In [ ]:
# nb25_react_pattern_minimal.ipynb
# ReAct Pattern: Thought → Action → Observation → Answer


# Cell1:  Shared Cache Bootstrap
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

In [ ]:
# Cell 2: Import & Setup
import json
import re
import ast
from typing import Dict, List, Any, Optional, Tuple
from transformers import AutoTokenizer, AutoModelForCausalLM
from duckduckgo_search import DDGS
import requests
from pathlib import Path


# Simple LLM Adapter
class LLMAdapter:
    def __init__(self, model_id="Qwen/Qwen2.5-7B-Instruct"):
        print(f"Loading {model_id}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            torch_dtype="auto",
            load_in_4bit=True,  # Low VRAM
        )

    def generate(self, prompt: str, max_new_tokens=512, temperature=0.7) -> str:
        inputs = self.tokenizer(
            prompt, return_tensors="pt", truncation=True, max_length=2048
        )
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
            )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract only the new generation
        prompt_length = len(
            self.tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)
        )
        return response[prompt_length:].strip()


# Initialize LLM
llm = LLMAdapter()

In [ ]:
# Cell 3: Tool Registry & Schema
class ToolRegistry:
    def __init__(self):
        self.tools = {}
        self.register_default_tools()

    def register_tool(self, name: str, func, description: str, schema: dict):
        """Register a tool with its function and schema"""
        self.tools[name] = {
            "function": func,
            "description": description,
            "schema": schema,
        }

    def get_tools_description(self) -> str:
        """Get formatted description of all available tools"""
        desc = "Available tools:\n"
        for name, tool in self.tools.items():
            desc += f"- {name}: {tool['description']}\n"
            desc += f"  Schema: {json.dumps(tool['schema'])}\n"
        return desc

    def execute_tool(self, name: str, args: dict) -> str:
        """Execute a tool with given arguments"""
        if name not in self.tools:
            return f"Error: Tool '{name}' not found"

        try:
            return self.tools[name]["function"](**args)
        except Exception as e:
            return f"Error executing {name}: {str(e)}"

    def register_default_tools(self):
        """Register default tools"""

        # Calculator tool
        def safe_calculator(expression: str) -> str:
            """Safe arithmetic calculator"""
            try:
                # Parse as AST to ensure safety
                node = ast.parse(expression, mode="eval")

                # Only allow specific node types
                allowed_nodes = (
                    ast.Expression,
                    ast.Num,
                    ast.BinOp,
                    ast.UnaryOp,
                    ast.Add,
                    ast.Sub,
                    ast.Mult,
                    ast.Div,
                    ast.Pow,
                    ast.USub,
                    ast.UAdd,
                    ast.Constant,
                )

                for node_item in ast.walk(node):
                    if not isinstance(node_item, allowed_nodes):
                        return f"Error: Unsafe operation in expression"

                result = eval(compile(node, "<string>", "eval"))
                return f"Result: {result}"
            except Exception as e:
                return f"Error: {str(e)}"

        self.register_tool(
            "calculator",
            safe_calculator,
            "Evaluate arithmetic expressions safely",
            {"expression": "string (e.g., '2+3*4')"},
        )

        # Web search tool
        def web_search(query: str, max_results: int = 3) -> str:
            """Search the web using DuckDuckGo"""
            try:
                with DDGS() as ddgs:
                    results = list(ddgs.text(query, max_results=max_results))
                    if not results:
                        return "No search results found"

                    output = f"Search results for '{query}':\n"
                    for i, result in enumerate(results, 1):
                        output += f"{i}. {result['title']}\n"
                        output += f"   {result['body'][:200]}...\n"
                        output += f"   URL: {result['href']}\n\n"
                    return output
            except Exception as e:
                return f"Search error: {str(e)}"

        self.register_tool(
            "web_search",
            web_search,
            "Search the web for information",
            {"query": "string", "max_results": "integer (optional, default 3)"},
        )

        # File lookup tool
        def file_lookup(filename: str) -> str:
            """Look up content in allowed directories"""
            try:
                # Whitelist allowed paths
                allowed_dirs = ["data", "outs", "configs"]
                file_path = Path(filename)

                # Check if path is within allowed directories
                allowed = any(str(file_path).startswith(d) for d in allowed_dirs)
                if not allowed:
                    return f"Error: Access denied to {filename}"

                if file_path.exists() and file_path.is_file():
                    content = file_path.read_text(encoding="utf-8")[
                        :1000
                    ]  # Limit content
                    return f"File content:\n{content}"
                else:
                    return f"Error: File {filename} not found"
            except Exception as e:
                return f"File lookup error: {str(e)}"

        self.register_tool(
            "file_lookup",
            file_lookup,
            "Look up file content in data/outs/configs directories",
            {"filename": "string (relative path)"},
        )


# Initialize tool registry
tools = ToolRegistry()

In [ ]:
# Cell 4: ReAct Prompt Template
REACT_PROMPT_TEMPLATE = """You are a helpful AI assistant that can use tools to answer questions.
You must follow this exact format for each step:

Thought: [Your reasoning about what to do next]
Action: {"tool": "tool_name", "args": {"arg1": "value1"}}
Observation: [Tool output will be provided here]

Continue this pattern until you have enough information to answer.
When ready to answer, use:
Answer: [Your final response]

{tools_description}

Question: {question}

Begin:
"""


def parse_action(text: str) -> Optional[Tuple[str, dict]]:
    """Parse action JSON from LLM output"""
    try:
        # Look for JSON in the Action line
        action_match = re.search(r"Action:\s*(\{.*?\})", text, re.DOTALL)
        if action_match:
            action_json = action_match.group(1)
            action_data = json.loads(action_json)
            tool_name = action_data.get("tool")
            args = action_data.get("args", {})
            return tool_name, args
    except (json.JSONDecodeError, AttributeError) as e:
        pass
    return None


def extract_answer(text: str) -> Optional[str]:
    """Extract final answer from ReAct output"""
    answer_match = re.search(r"Answer:\s*(.*?)(?:\n|$)", text, re.DOTALL)
    if answer_match:
        return answer_match.group(1).strip()
    return None

In [ ]:
# Cell 5: Core ReAct Loop
class ReActAgent:
    def __init__(self, llm: LLMAdapter, tools: ToolRegistry, max_iterations=5):
        self.llm = llm
        self.tools = tools
        self.max_iterations = max_iterations

    def run(self, question: str) -> str:
        """Run ReAct loop for a given question"""

        # Initialize prompt
        prompt = REACT_PROMPT_TEMPLATE.format(
            tools_description=self.tools.get_tools_description(), question=question
        )

        conversation = prompt

        for iteration in range(self.max_iterations):
            print(f"\n--- Iteration {iteration + 1} ---")

            # Generate response from LLM
            response = self.llm.generate(
                conversation, max_new_tokens=256, temperature=0.3
            )
            conversation += response

            print(f"LLM Response: {response[:200]}...")

            # Check if we have a final answer
            final_answer = extract_answer(response)
            if final_answer:
                print(f"Final answer found: {final_answer}")
                return final_answer

            # Parse action
            action_result = parse_action(response)
            if action_result:
                tool_name, args = action_result
                print(f"Executing tool: {tool_name} with args: {args}")

                # Execute tool
                observation = self.tools.execute_tool(tool_name, args)

                # Add observation to conversation
                conversation += f"\nObservation: {observation}\n"
                print(f"Observation: {observation[:100]}...")
            else:
                # No valid action found, prompt for continuation
                conversation += "\nPlease provide a valid Action in JSON format or your final Answer.\n"

        return "Sorry, I couldn't solve this problem within the maximum iterations."


# Initialize agent
agent = ReActAgent(llm, tools, max_iterations=5)

In [ ]:
# Cell 6: Error Handling & Retry
class RobustReActAgent(ReActAgent):
    def __init__(
        self, llm: LLMAdapter, tools: ToolRegistry, max_iterations=5, max_retries=2
    ):
        super().__init__(llm, tools, max_iterations)
        self.max_retries = max_retries

    def run_with_retry(self, question: str) -> str:
        """Run ReAct with retry mechanism"""

        for attempt in range(self.max_retries + 1):
            try:
                print(f"\n=== Attempt {attempt + 1} ===")
                result = self.run(question)

                # Check if result is meaningful (not just error message)
                if not result.startswith("Sorry, I couldn't"):
                    return result

            except Exception as e:
                print(f"Attempt {attempt + 1} failed: {str(e)}")
                if attempt == self.max_retries:
                    return f"Error: Maximum retries exceeded. Last error: {str(e)}"

        return "Error: All attempts failed to produce a valid answer."


# Initialize robust agent
robust_agent = RobustReActAgent(llm, tools, max_iterations=3, max_retries=1)

In [ ]:
# Cell 7: Smoke Test - Multi-step Problem
print("=== Smoke Test: Multi-step Problem ===")

# Test question that requires multiple tools
test_question = (
    "What is 15 * 23, and can you search for information about the ReAct pattern in AI?"
)

print(f"Question: {test_question}")
print(
    f"Expected: Should use calculator for 15*23=345, then search for ReAct pattern info"
)

# Run the test
result = robust_agent.run_with_retry(test_question)
print(f"\n=== Final Result ===")
print(result)

# Simple validation
print(f"\n=== Validation ===")
contains_calculation = "345" in result or "15" in result and "23" in result
contains_search_info = any(
    word in result.lower() for word in ["react", "reasoning", "action", "thought"]
)

print(f"✓ Contains calculation result: {contains_calculation}")
print(f"✓ Contains search information: {contains_search_info}")
print(f"✓ Overall success: {contains_calculation or contains_search_info}")

In [ ]:
# Cell 8: What we built / Next steps
print(
    """
=== What we built ===
✓ Minimal ReAct Agent: Thought → Action → Observation → Answer
✓ Tool Registry: Unified tool management and execution
✓ Error Handling: Retry mechanism for failed attempts
✓ JSON Parsing: Robust action extraction from LLM output
✓ Safety: Whitelisted tools and safe execution

=== Key Components ===
• LLMAdapter: Simple wrapper for Qwen2.5-7B with 4-bit loading
• ToolRegistry: Calculator, web search, file lookup
• ReActAgent: Core reasoning loop with iteration limits
• Action Parser: Extract tool calls from natural language

=== Pitfalls ===
• LLM may not follow JSON format exactly - need robust parsing
• Tool execution can fail - always handle exceptions
• Infinite loops possible - set max iterations
• Context window overflow - manage conversation length

=== Next Steps ===
• Add more sophisticated tools (code execution, API calls)
• Implement memory/context compression for long conversations
• Add tool validation and security checks
• Integrate with RAG for knowledge-based reasoning
• Build multi-agent orchestration on this foundation

=== When to use this ===
• Multi-step problems requiring tool use
• Situations where reasoning steps should be hidden from user
• Building blocks for more complex agent systems
• Prototyping tool-augmented AI applications
"""
)